In [ ]:
from selenium import webdriver
import subprocess
import chromedriver_autoinstaller
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import chromedriver_autoinstaller
    
from selenium.webdriver.common.keys import Keys

import time
import datetime as dt
import schedule

import pandas as pd
import numpy as np
from pprint import pprint
import urllib.request
import json
import glob
import sys
import os

import warnings
warnings.filterwarnings(action='ignore')

pd.set_option('display.max_columns', 250)
pd.set_option('display.max_rows', 250)
pd.set_option('display.width', 100)

pd.options.display.float_format = '{:.2f}'.format

class NaverDataLabOpenAPI():
    """
    네이버 데이터랩 오픈 API 컨트롤러 클래스
    """

    def __init__(self, client_id, client_secret):
        """
        인증키 설정 및 검색어 그룹 초기화
        """
        self.client_id = client_id
        self.client_secret = client_secret
        self.keywordGroups = []
        self.url = "https://openapi.naver.com/v1/datalab/search"

    def add_keyword_groups(self, group_dict):
        """
        검색어 그룹 추가
        """

        keyword_gorup = {
            'groupName': group_dict['groupName'],
            'keywords': group_dict['keywords']
        }
        
        self.keywordGroups.append(keyword_gorup)
#         print(f">>> Num of keywordGroups: {len(self.keywordGroups)}")
        
    def get_data(self, startDate, endDate, timeUnit, device, ages, gender):
        """
        요청 결과 반환
        timeUnit - 'date', 'week', 'month'
        device - None, 'pc', 'mo'
        ages = [], ['1' ~ '11']
        gender = None, 'm', 'f'
        """

        # Request body
        body = json.dumps({
            "startDate": startDate,
            "endDate": endDate,
            "timeUnit": timeUnit,
            "keywordGroups": self.keywordGroups,
            "device": device,
            "ages": ages,
            "gender": gender
        }, ensure_ascii=False)
        
        # Results
        request = urllib.request.Request(self.url)
        request.add_header("X-Naver-Client-Id",self.client_id)
        request.add_header("X-Naver-Client-Secret",self.client_secret)
        request.add_header("Content-Type","application/json")
        response = urllib.request.urlopen(request, data=body.encode("utf-8"))
        rescode = response.getcode()
        if(rescode==200):
            # Json Result
            result = json.loads(response.read())
            
            period = []
            ratio = []
            title = []
#             pprint(result)
            if result['results'][0]['data']:
                for val in result['results']:
                    period.append(val['data'][-1]['period'])
                    ratio.append(val['data'][-1]['ratio'])
                    title.append(val['title'])
            
            df = pd.DataFrame({"title" : title, 'period' : period, "ratio" : ratio})
            
        else:
            print("Error Code:" + rescode)
            
        return df

keyword_group_set = {
    'keyword_group_1': {'groupName': "애플", 'keywords': ["애플"]},
}
def run():
    print("시작시작시작")
    try:
        subprocess.Popen(r'C:\Program Files\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp1"')  # 디버거 크롬 구동
    except:
        subprocess.Popen(r'C:\Program Files (x86)\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp1"')  # 디버거 크롬 구동
    option = Options()
    option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
    # option.add_argument("--headless")

    chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
    try:
        driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
    except:
        chromedriver_autoinstaller.install('./')
        driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)

    ######################
    #플라스틱제로 시작
    #####크롤링
    print('plastic start')
    driver.get('https://blackkiwi.net/service/keyword-analysis?keyword=%ED%94%8C%EB%9D%BC%EC%8A%A4%ED%8B%B1%20%EC%A0%9C%EB%A1%9C&platform=naver')
    time.sleep(10) #혹시 모를 에러 방지를 위한 적절한 wait 넣어주기
    title=driver.find_elements(By.CSS_SELECTOR,'table a')
    title_l=[]
    for i in range(20):
        title_l.append(title[i].text)
#         print(title_l[i])
    driver.implicitly_wait(10)
    driver.get('http://surffing.net/MainSeoSearch.do')
    search=driver.find_element(By.ID,'saerchKeyword')
    search.send_keys('플라스틱 제로')
    search.send_keys(Keys.ENTER)
    title2=driver.find_elements(By.CSS_SELECTOR,'tr>td.center:nth-child(1)')
    title_b=[]
    
    for i in range(len(title2)):
        title_b.append(title2[i].text)
#         print(title_b[i])
    
    title_l.extend(title_b)
    list_a=[]
    for i in range(len(title_l)):
            list_a.append(title_l[i].replace(" ", ""))
    li_set=set(list_a)
    list_a=list(li_set)
    
    today = dt.date.today()
    if(today.weekday()==0):
        monday=dt.date.today()-dt.timedelta(+7)
        print(monday)
    else:
        monday=today - dt.timedelta(days=today.weekday())
    days=today.weekday()
    end=monday.strftime('%Y-%m-%d')
    print(end)
    th=dt.timedelta(days=30)
    start=monday-th
    print(start)
    start=start.strftime('%Y-%m-%d')
        
    #####크롤링end
    cnt=0
    while(True):
        if len(list_a)>0 :
            for idx, (key, val) in enumerate(keyword_group_set.items()):
                val['groupName'] = list_a[idx]
                val['keywords'] = [list_a[idx]]
            del list_a[:1]
            cnt+=1
        else:
            break
    #     pprint(keyword_group_set)

            # API 인증 정보 설정
        client_id = "xTsOEpqU869lYEaqyaU6"
        client_secret = "kkp_ZP2j9T"

        # 요청 파라미터 설정
        startDate = start
        endDate = end
        timeUnit = 'week'
        device = ''
        ages = []
        gender = ''

        # 데이터 프레임 정의
        naver = NaverDataLabOpenAPI(client_id=client_id, client_secret=client_secret)

        naver.add_keyword_groups(keyword_group_set['keyword_group_1'])

        df = naver.get_data(startDate, endDate, timeUnit, device, ages, gender)

        if(cnt>1):
            df1=pd.concat([df1,df])
        else:
            df1=df
#     print(df1)
    #검색주간과 같은 데이터만 추출
    df1=df1[df1['period']==end]
    #ratio높은순으로 정렬
    df2=df1.sort_values('ratio',ascending=False)
    #ratio 삭제
    df2=df2.drop([df2.columns[2]],axis=1)
    #인덱싱해주기
    df2.reset_index(drop=True,inplace=True)
    #10위까지 저장
    data=df2.loc[:9]
    print(data)
    
    #db에 데이터 저장
    import cx_Oracle

    dsn = cx_Oracle.makedsn("project-db-stu.ddns.net", 1524, service_name = "XE") # 오라클 주소
    connection = cx_Oracle.connect(user="gjai_4_2_0822", password="smhrd2", dsn=dsn, encoding="UTF-8") # 오라클 접속
    cur = connection.cursor() # 실행 결과 데이터를 담을 메모리 객체
    #dataframe 변환
    df_list = data.to_records(index=False)
    df_list = data.values.tolist()
    #쿼리
    cur.execute("TRUNCATE TABLE plastic_search")
    cur.executemany("INSERT INTO plastic_search (title, day) VALUES (:1,:2)",
            df_list, batcherrors = True)
    cur.execute("commit")
    print('plastic success commit')
    
    
    ######################
    #플라스틱제로 끝
    ######################
    #비건 시작
    
    
    print('vegan start')
    driver.get('https://blackkiwi.net/service/keyword-analysis?keyword=%EB%B9%84%EA%B1%B4&platform=naver')
    time.sleep(10) #혹시 모를 에러 방지를 위한 적절한 wait 넣어주기
    title=driver.find_elements(By.CSS_SELECTOR,'table a')
    title_l=[]
    for i in range(20):
        title_l.append(title[i].text)
#         print(title_l[i])
    driver.implicitly_wait(10)
    driver.get('http://surffing.net/MainSeoSearch.do')
    search=driver.find_element(By.ID,'saerchKeyword')
    search.send_keys('비건')
    search.send_keys(Keys.ENTER)
    title2=driver.find_elements(By.CSS_SELECTOR,'tr>td.center:nth-child(1)')
    title_b=[]

    for i in range(len(title2)):
        title_b.append(title2[i].text)
#         print(title_b[i])
        
    ##채식
    driver.get('https://blackkiwi.net/service/keyword-analysis?keyword=%EC%B1%84%EC%8B%9D&platform=naver')
    time.sleep(10) #혹시 모를 에러 방지를 위한 적절한 wait 넣어주기
    title3=driver.find_elements(By.CSS_SELECTOR,'table a')
    title_3=[]
    for i in range(20):
        title_3.append(title3[i].text)
#         print(title_3[i])
    driver.implicitly_wait(10)
    driver.get('http://surffing.net/MainSeoSearch.do')
    search=driver.find_element(By.ID,'saerchKeyword')
    search.send_keys('채식')
    search.send_keys(Keys.ENTER)
    title4=driver.find_elements(By.CSS_SELECTOR,'tr>td.center:nth-child(1)')
    title_4=[]

    for i in range(len(title4)):
        title_4.append(title4[i].text)
#         print(title_4[i])
    
    title_l.extend(title_b)
    title_l.extend(title_3)
    title_l.extend(title_4)
    list_a=[]
    for i in range(len(title_l)):
            list_a.append(title_l[i].replace(" ", ""))
    li_set=set(list_a)
    list_a=list(li_set)
    
    cnt=0
    while(True):
        if len(list_a)>0 :
            for idx, (key, val) in enumerate(keyword_group_set.items()):
                val['groupName'] = list_a[idx]
                val['keywords'] = [list_a[idx]]
            del list_a[:1]
            cnt+=1
        else:
            break
    #     pprint(keyword_group_set)

            # API 인증 정보 설정
        client_id = "xTsOEpqU869lYEaqyaU6"
        client_secret = "kkp_ZP2j9T"

        # 요청 파라미터 설정
        startDate = start
        endDate = end
        timeUnit = 'week'
        device = ''
        ages = []
        gender = ''

        # 데이터 프레임 정의
        naver = NaverDataLabOpenAPI(client_id=client_id, client_secret=client_secret)

        naver.add_keyword_groups(keyword_group_set['keyword_group_1'])

        df = naver.get_data(startDate, endDate, timeUnit, device, ages, gender)

        if(cnt>1):
            df1=pd.concat([df1,df])
        else:
            df1=df
#     print(df1)
    #검색주간과 같은 데이터만 추출
    df1=df1[df1['period']==end]
    #ratio높은순으로 정렬
    df2=df1.sort_values('ratio',ascending=False)
    #ratio 삭제
    df2=df2.drop([df2.columns[2]],axis=1)
    #인덱싱해주기
    df2.reset_index(drop=True,inplace=True)
    #10위까지 저장
    data=df2.loc[:9]
    print(data)
    
    #db에 데이터 저장
    import cx_Oracle

    dsn = cx_Oracle.makedsn("project-db-stu.ddns.net", 1524, service_name = "XE") # 오라클 주소
    connection = cx_Oracle.connect(user="gjai_4_2_0822", password="smhrd2", dsn=dsn, encoding="UTF-8") # 오라클 접속
    cur = connection.cursor() # 실행 결과 데이터를 담을 메모리 객체
    #dataframe 변환
    df_list = data.to_records(index=False)
    df_list = data.values.tolist()
    #쿼리
    cur.execute("TRUNCATE TABLE vegan_search")
    cur.executemany("INSERT INTO vegan_search (title, day) VALUES (:1,:2)",
            df_list, batcherrors = True)
    cur.execute("commit")
    print('vegan commit')
    
    ######################
    #비건 끝
    ######################
    #제로웨이스트 시작
    print('zerowaste start')
    driver.get('https://blackkiwi.net/service/keyword-analysis?keyword=%EC%A0%9C%EB%A1%9C%EC%9B%A8%EC%9D%B4%EC%8A%A4%ED%8A%B8&platform=naver')
    time.sleep(10) #혹시 모를 에러 방지를 위한 적절한 wait 넣어주기
    title=driver.find_elements(By.CSS_SELECTOR,'table a')
    title_l=[]
    for i in range(20):
        title_l.append(title[i].text)
#         print(title_l[i])
    driver.implicitly_wait(10)
    driver.get('http://surffing.net/MainSeoSearch.do')
    search=driver.find_element(By.ID,'saerchKeyword')
    search.send_keys('제로웨이스트')
    search.send_keys(Keys.ENTER)
    title2=driver.find_elements(By.CSS_SELECTOR,'tr>td.center:nth-child(1)')
    title_b=[]

    for i in range(len(title2)):
        title_b.append(title2[i].text)
#         print(title_b[i])
    
    title_l.extend(title_b)
    list_a=[]
    for i in range(len(title_l)):
            list_a.append(title_l[i].replace(" ", ""))
    li_set=set(list_a)
    list_a=list(li_set)
    

        
    #####크롤링end
    cnt=0
    while(True):
        if len(list_a)>0 :
            for idx, (key, val) in enumerate(keyword_group_set.items()):
                val['groupName'] = list_a[idx]
                val['keywords'] = [list_a[idx]]
            del list_a[:1]
            cnt+=1
        else:
            break
    #     pprint(keyword_group_set)

            # API 인증 정보 설정
        client_id = "xTsOEpqU869lYEaqyaU6"
        client_secret = "kkp_ZP2j9T"

        # 요청 파라미터 설정
        startDate = start
        endDate = end
        timeUnit = 'week'
        device = ''
        ages = []
        gender = ''

        # 데이터 프레임 정의
        naver = NaverDataLabOpenAPI(client_id=client_id, client_secret=client_secret)

        naver.add_keyword_groups(keyword_group_set['keyword_group_1'])

        df = naver.get_data(startDate, endDate, timeUnit, device, ages, gender)

        if(cnt>1):
            df1=pd.concat([df1,df])
        else:
            df1=df
#     print(df1)

    #드라이버닫기
    driver.close()
    #검색주간과 같은 데이터만 추출
    df1=df1[df1['period']==end]
    #ratio높은순으로 정렬
    df2=df1.sort_values('ratio',ascending=False)
    #ratio 삭제
    df2=df2.drop([df2.columns[2]],axis=1)
    #인덱싱해주기
    #인덱싱해주기
    df2.reset_index(drop=True,inplace=True)
    #10위까지 저장
    data=df2.loc[:9]
    print(data)
    
     #db에 데이터 저장
    import cx_Oracle

    dsn = cx_Oracle.makedsn("project-db-stu.ddns.net", 1524, service_name = "XE") # 오라클 주소
    connection = cx_Oracle.connect(user="gjai_4_2_0822", password="smhrd2", dsn=dsn, encoding="UTF-8") # 오라클 접속
    cur = connection.cursor() # 실행 결과 데이터를 담을 메모리 객체
    #dataframe 변환
    df_list = data.to_records(index=False)
    df_list = data.values.tolist()
    #쿼리
    cur.execute("TRUNCATE TABLE zerowaste_search")
    cur.executemany("INSERT INTO zerowaste_search (title, day) VALUES (:1,:2)",
            df_list, batcherrors = True)
    cur.execute("commit")
    print('zerowaste commit')
    
    
    
    
    
    cur.close()
    connection.close()

    
    
# step3.실행 주기 설정
# schedule.every().tuesday.at("02:00").do(run)
schedule.every().day.at("21:24").do(run)

# step4.스캐쥴 시작
while True:
    schedule.run_pending()
    time.sleep(1)

시작시작시작
plastic start
2022-08-22
2022-08-22
2022-07-23
{'endDate': '2022-08-28',
 'results': [{'data': [{'period': '2022-07-18', 'ratio': 100},
                       {'period': '2022-07-25', 'ratio': 79.48717},
                       {'period': '2022-08-01', 'ratio': 54.87179},
                       {'period': '2022-08-08', 'ratio': 57.82051},
                       {'period': '2022-08-15', 'ratio': 60.64102},
                       {'period': '2022-08-22', 'ratio': 70.51282}],
              'keywords': ['생분해플라스틱'],
              'title': '생분해플라스틱'}],
 'startDate': '2022-07-18',
 'timeUnit': 'week'}
{'endDate': '2022-08-28',
 'results': [{'data': [{'period': '2022-07-18', 'ratio': 72.87234},
                       {'period': '2022-07-25', 'ratio': 61.70212},
                       {'period': '2022-08-01', 'ratio': 56.91489},
                       {'period': '2022-08-08', 'ratio': 80.85106},
                       {'period': '2022-08-15', 'ratio': 77.12765},
                       {'p

{'endDate': '2022-08-28',
 'results': [{'data': [{'period': '2022-07-18', 'ratio': 68.51361},
                       {'period': '2022-07-25', 'ratio': 100},
                       {'period': '2022-08-01', 'ratio': 75.64296},
                       {'period': '2022-08-08', 'ratio': 96.65279},
                       {'period': '2022-08-15', 'ratio': 59.05824},
                       {'period': '2022-08-22', 'ratio': 79.236}],
              'keywords': ['고체치약'],
              'title': '고체치약'}],
 'startDate': '2022-07-18',
 'timeUnit': 'week'}
{'endDate': '2022-08-28',
 'results': [{'data': [], 'keywords': ['도전!플라스틱제로'], 'title': '도전!플라스틱제로'}],
 'startDate': '2022-07-18',
 'timeUnit': 'week'}
{'endDate': '2022-08-28',
 'results': [{'data': [{'period': '2022-07-18', 'ratio': 89.85507},
                       {'period': '2022-07-25', 'ratio': 28.62318},
                       {'period': '2022-08-01', 'ratio': 27.89855},
                       {'period': '2022-08-08', 'ratio': 33.69565},
    

{'endDate': '2022-08-28',
 'results': [{'data': [{'period': '2022-07-18', 'ratio': 68.94865},
                       {'period': '2022-07-25', 'ratio': 78.2396},
                       {'period': '2022-08-01', 'ratio': 100},
                       {'period': '2022-08-08', 'ratio': 91.19804},
                       {'period': '2022-08-15', 'ratio': 81.41809},
                       {'period': '2022-08-22', 'ratio': 84.10757}],
              'keywords': ['채식부페'],
              'title': '채식부페'}],
 'startDate': '2022-07-18',
 'timeUnit': 'week'}
{'endDate': '2022-08-28',
 'results': [{'data': [{'period': '2022-07-18', 'ratio': 62.0155},
                       {'period': '2022-07-25', 'ratio': 44.18604},
                       {'period': '2022-08-01', 'ratio': 82.94573},
                       {'period': '2022-08-08', 'ratio': 100},
                       {'period': '2022-08-15', 'ratio': 81.39534},
                       {'period': '2022-08-22', 'ratio': 82.94573}],
              'keywords'

{'endDate': '2022-08-28',
 'results': [{'data': [{'period': '2022-07-18', 'ratio': 95.36423},
                       {'period': '2022-07-25', 'ratio': 83.77483},
                       {'period': '2022-08-01', 'ratio': 100},
                       {'period': '2022-08-08', 'ratio': 85.76158},
                       {'period': '2022-08-15', 'ratio': 82.78145},
                       {'period': '2022-08-22', 'ratio': 98.34437}],
              'keywords': ['채식한끼'],
              'title': '채식한끼'}],
 'startDate': '2022-07-18',
 'timeUnit': 'week'}
{'endDate': '2022-08-28',
 'results': [{'data': [{'period': '2022-07-18', 'ratio': 100},
                       {'period': '2022-07-25', 'ratio': 91.92399},
                       {'period': '2022-08-01', 'ratio': 82.66033},
                       {'period': '2022-08-08', 'ratio': 87.17339},
                       {'period': '2022-08-15', 'ratio': 90.97387},
                       {'period': '2022-08-22', 'ratio': 84.79809}],
              'keyword

{'endDate': '2022-08-28',
 'results': [{'data': [{'period': '2022-07-18', 'ratio': 16.66666},
                       {'period': '2022-07-25', 'ratio': 25},
                       {'period': '2022-08-01', 'ratio': 30},
                       {'period': '2022-08-08', 'ratio': 38.33333},
                       {'period': '2022-08-15', 'ratio': 53.33333},
                       {'period': '2022-08-22', 'ratio': 100}],
              'keywords': ['김장체험'],
              'title': '김장체험'}],
 'startDate': '2022-07-18',
 'timeUnit': 'week'}
{'endDate': '2022-08-28',
 'results': [{'data': [{'period': '2022-07-18', 'ratio': 93.07228},
                       {'period': '2022-07-25', 'ratio': 82.83132},
                       {'period': '2022-08-01', 'ratio': 78.31325},
                       {'period': '2022-08-08', 'ratio': 77.40963},
                       {'period': '2022-08-15', 'ratio': 92.16867},
                       {'period': '2022-08-22', 'ratio': 100}],
              'keywords': ['비건백'],

{'endDate': '2022-08-28',
 'results': [{'data': [{'period': '2022-07-18', 'ratio': 70.86441},
                       {'period': '2022-07-25', 'ratio': 78.43352},
                       {'period': '2022-08-01', 'ratio': 83.77577},
                       {'period': '2022-08-08', 'ratio': 81.29662},
                       {'period': '2022-08-15', 'ratio': 81.05528},
                       {'period': '2022-08-22', 'ratio': 100}],
              'keywords': ['제철음식'],
              'title': '제철음식'}],
 'startDate': '2022-07-18',
 'timeUnit': 'week'}
{'endDate': '2022-08-28',
 'results': [{'data': [{'period': '2022-07-18', 'ratio': 100},
                       {'period': '2022-07-25', 'ratio': 92.03601},
                       {'period': '2022-08-01', 'ratio': 84.07202},
                       {'period': '2022-08-08', 'ratio': 85.73407},
                       {'period': '2022-08-15', 'ratio': 96.8144},
                       {'period': '2022-08-22', 'ratio': 78.87811}],
              'keywords

              'title': '제주도수학여행'}],
 'startDate': '2022-07-18',
 'timeUnit': 'week'}
{'endDate': '2022-08-28',
 'results': [{'data': [{'period': '2022-07-18', 'ratio': 55.55555},
                       {'period': '2022-07-25', 'ratio': 66.66666},
                       {'period': '2022-08-01', 'ratio': 46.29629},
                       {'period': '2022-08-08', 'ratio': 27.77777},
                       {'period': '2022-08-15', 'ratio': 68.51851},
                       {'period': '2022-08-22', 'ratio': 100}],
              'keywords': ['제로웨이스트비누'],
              'title': '제로웨이스트비누'}],
 'startDate': '2022-07-18',
 'timeUnit': 'week'}
{'endDate': '2022-08-28',
 'results': [{'data': [{'period': '2022-07-18', 'ratio': 97.9452},
                       {'period': '2022-07-25', 'ratio': 84.01826},
                       {'period': '2022-08-01', 'ratio': 76.48401},
                       {'period': '2022-08-08', 'ratio': 75.34246},
                       {'period': '2022-08-15', 'ratio': 81.65

{'endDate': '2022-08-28',
 'results': [{'data': [{'period': '2022-07-18', 'ratio': 70.16528},
                       {'period': '2022-07-25', 'ratio': 49.83471},
                       {'period': '2022-08-01', 'ratio': 52.85123},
                       {'period': '2022-08-08', 'ratio': 60.86776},
                       {'period': '2022-08-15', 'ratio': 59.00826},
                       {'period': '2022-08-22', 'ratio': 100}],
              'keywords': ['친환경제품'],
              'title': '친환경제품'}],
 'startDate': '2022-07-18',
 'timeUnit': 'week'}
{'endDate': '2022-08-28',
 'results': [{'data': [{'period': '2022-07-18', 'ratio': 100},
                       {'period': '2022-07-25', 'ratio': 75.2561},
                       {'period': '2022-08-01', 'ratio': 72.63987},
                       {'period': '2022-08-08', 'ratio': 71.21355},
                       {'period': '2022-08-15', 'ratio': 83.50669},
                       {'period': '2022-08-22', 'ratio': 86.19385}],
              'keywor